# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **New York City Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **New York City Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Always backup your solution in a separate file or your personal computer.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# display(trips.take(10))

Trips dataset loaded!
 ---
 Size: XXL
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-XXL-delta/
 Number of trips (dataset rows): 132,396,785

In [0]:
print_count(trips)

Row count: 132,396,785

In [0]:
# dataset schemas
trips.printSchema()
zone_names.printSchema()

root
-- index: long (nullable = true)
-- VendorID: double (nullable = true)
-- tpep_pickup_datetime: string (nullable = true)
-- tpep_dropoff_datetime: string (nullable = true)
-- passenger_count: double (nullable = true)
-- trip_distance: double (nullable = true)
-- RatecodeID: double (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- PULocationID: long (nullable = true)
-- DOLocationID: long (nullable = true)
-- payment_type: double (nullable = true)
-- fare_amount: double (nullable = true)
-- extra: double (nullable = true)
-- mta_tax: double (nullable = true)
-- tip_amount: double (nullable = true)
-- tolls_amount: double (nullable = true)
-- improvement_surcharge: double (nullable = true)
-- total_amount: double (nullable = true)
-- congestion_surcharge: double (nullable = true)
-- cab_type: string (nullable = true)
-- lpep_pickup_datetime: string (nullable = true)
-- lpep_dropoff_datetime: string (nullable = true)
-- ehail_fee: double (nullable = true)
-- trip_type: double (nullable = true)

root
-- LocationID: long (nullable = true)
-- Borough: string (nullable = true)
-- Zone: string (nullable = true)
-- service_zone: string (nullable = true)

## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.438 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

Before running task 1.1 the first thing I am doing is selecting only the columns that are needed for the task, as this makes computattion faster, and also visualisation of the results is clearer. Now for task 1.1 I am using filter and I am filtering out the zero distance and no passenger records as required. Because filter keeps what is inside the filter function, I need to reverse the conditions in order to keep the records that I need.

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
  #select only specific columns 
    cols = ['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']
    df = df.select(*cols)
    df = df.filter((df.trip_distance != 0) & ((df.passenger_count != 0) | (df.total_amount > 0)))

    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
#display(trips_11.take(10))

Row count: 130,901,064

For task 1.2 the first thing that I am doing is to calculate the median of the trip distance and total amount columns by using the percentile approximation function. The outcome is a dataframe so I am using the collect query in order to extract the integer value and then be able to perform the numerical calculations. After having calculated the median I am creating two new columns (the `z_trip_distance` and `z_total_amount`) and I am storing the numerator (x(i) −median(x)) in each of them. After that I am calculating the absolute value of the median of the new created columns (median(|x(i)- median(x)|) and then I am changing the columns I created before by doing the zi calculation. Finally I am using filter (by reversing the condition again) in order to remove the outliers.

In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    median_trip_distance = df.agg(pf.percentile_approx('trip_distance', 0.5, 10000)).collect()[0][0]
    median_total_amount = df.agg(pf.percentile_approx('total_amount', 0.5, 10000)).collect()[0][0]
    df = df.withColumn('z_trip_distance', df.trip_distance - median_trip_distance)
    median_trip_distance_abs = df.agg(pf.percentile_approx( pf.abs(df.z_trip_distance), 0.5, 10000)).collect()[0][0]
    df = df.withColumn('z_trip_distance', df.z_trip_distance/(1.438*median_trip_distance_abs))
    
    df = df.withColumn('z_total_amount', df.total_amount - median_total_amount)
    median_total_amount_abs = df.agg(pf.percentile_approx( pf.abs(df.z_total_amount), 0.5, 10000)).collect()[0][0]
    df = df.withColumn('z_total_amount', df.z_total_amount/(1.438*median_total_amount_abs))
    
    df = df.filter((pf.abs(df.z_trip_distance) <= 3.5) & (pf.abs(df.z_total_amount) <= 3.5))

    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)

# display(trips_12.take(10))

Row count: 114,173,013

## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

Before doing task 2.1 I am droping the `z_trip_distance` and `z_total_amount columns` since these columns are not needed in the next task. Now for task 1.2 I am using join twice. Once to join the pickup location ID with the name of the location and then I am doing the same for dropoff location. In order to perform the second join I need to rename the zones data frame because otherwise the join is not working. I am using left join in this case. Even though inner join may be a bit faster, what is doing is that it joins two datasets on key columns, and where keys don’t match the rows get dropped from both datasets. However, here if keys don't match we want a null value, so it is better to use left join which assigns a null value if a match is not found.

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    cols = ['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']
    df = df.select(*cols) 
    df = df.join(zones_df, df.PULocationID == zones_df.LocationID, "left").select(df['*'], zones_df['Zone'].alias('zone_PU'))
    zones_2 = zones_df.select(zones_df['LocationID'].alias('loc'), zones_df['Zone'].alias('zo')).alias('zones_2')
    df = df.join(zones_2, df.DOLocationID == zones_2.loc, "left").select(df['*'], zones_2['zo'].alias('zone_DO'))
    
# input: output of task 1.2 and zone_names dataset
    
    return df 

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
# display(trips_21.take(10))

Row count: 114,173,013

For task 2.2 I am just creating a new column where I am dividing the `total_amount` column with the `trip_distance` column in order to get the `Unit_Profitability` column.

In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    df = df.withColumn('Unit_Profitability', df.total_amount/df.trip_distance)
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
# display(trips_22.take(10))

Row count: 114,173,013

## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

For task 3.1, I am grouping the dataset by `PULocationID` and `DOLocationID` in order to get all of the zone to zone trips that are present in the data set. The aggregation is used in order to get the interzonal information required.

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    df = df.groupBy('PULocationID','DOLocationID').agg(pf.count('DOLocationID').alias('trips_count'),pf.mean('Unit_Profitability').alias('Average_unit_profitability'),pf.sum('passenger_count').alias('total_passengers') )
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
# graph.take(15)
# display(graph.take(10))

Row count: 21,503

For task 3.2 I first join the dataframe created in task 3.1 with the zone name dataframe in order to match the location ID with the zone name, since I have to group by zone name. Again I am renaming the dataframe and the columns in order for join to work. After that I am just grouping by `zone_PU` and I am calculating the total number of trips, the average profitability and the total passenger volume for each zone as required. 

After that to get the top 10 of each category, I am sorting with respect to the specific column that I want and in descending order. Then I take the top 10. I am using `sort` instead of `orderBy` because it is more efficient when it comes to computational time.

In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    zones_3 = zones_df.select(zones_df['LocationID'].alias('l'), zones_df['Zone'].alias('z')).alias('zones_3')
    df = df.join(zones_3, df.PULocationID == zones_3.l, "left").select(df['*'], zones_3['z'].alias('zone_PU'))

    df = df.groupBy('zone_PU').agg(pf.sum('trips_count').alias('total_trips_count'),pf.avg('Average_unit_profitability').alias('Average_unit_profitability_1'),pf.sum('total_passengers').alias('sum_total_passengers') )
    
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    df_zones = df_zones.sort(df_zones.total_trips_count.desc())
    df_zones = df_zones.select('zone_PU','total_trips_count').take(10)
    # input: output of task 3.2
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    df_zones = df_zones.sort(df_zones.Average_unit_profitability_1.desc())
    df_zones = df_zones.select('zone_PU','Average_unit_profitability_1').take(10)
    # input: output of task 3.2
    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    df_zones = df_zones.sort(df_zones.sum_total_passengers.desc())
    df_zones = df_zones.select('zone_PU','sum_total_passengers').take(10)
    # input: output of task 3.2
    return df_zones

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

Here I am using display in order to get tables of the top 10 zones for each category.

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

zone_PU,total_trips_count
Upper East Side South,5324909
Upper East Side North,4887578
Midtown Center,4506176
Penn Station/Madison Sq West,4080020
Midtown East,3980593
Lincoln Square East,3556452
Murray Hill,3535870
Clinton East,3469096
Union Sq,3425946
Times Sq/Theatre District,3423217


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

zone_PU,Average_unit_profitability_1
Bay Terrace/Fort Totten,107.22830164260448
Jamaica Estates,100.41824020495821
East Flushing,86.74211243912654
Glen Oaks,86.24724792951676
Far Rockaway,63.57453278925868
JFK Airport,60.24622939280393
Bay Ridge,57.488534704097276
Prospect-Lefferts Gardens,55.93947319736854
Co-Op City,52.162772873050784
Newark Airport,46.37322094876997


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

zone_PU,sum_total_passengers
Upper East Side South,7940953.0
Upper East Side North,7292727.0
Midtown Center,6889769.0
Penn Station/Madison Sq West,6099400.0
Midtown East,5984207.0
Times Sq/Theatre District,5422942.0
Lincoln Square East,5422590.0
Clinton East,5338571.0
Murray Hill,5293057.0
Union Sq,5219701.0


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'
WITH_TASK_12 = False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

Trips dataset loaded!
 ---
 Size: XXL
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-XXL-delta/
 Number of trips (dataset rows): 132,396,785

In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[36]: [[Row(zone_PU='Upper East Side South', total_trips_count=5547011),
 Row(zone_PU='Upper East Side North', total_trips_count=5116480),
 Row(zone_PU='Midtown Center', total_trips_count=4874992),
 Row(zone_PU='Penn Station/Madison Sq West', total_trips_count=4366447),
 Row(zone_PU='Midtown East', total_trips_count=4307043),
 Row(zone_PU='Times Sq/Theatre District', total_trips_count=3844450),
 Row(zone_PU='Murray Hill', total_trips_count=3813411),
 Row(zone_PU='Clinton East', total_trips_count=3780338),
 Row(zone_PU='Lincoln Square East', total_trips_count=3738256),
 Row(zone_PU='JFK Airport', total_trips_count=3697885)],
 [Row(zone_PU='Newark Airport', Average_unit_profitability_1=75.19307598426424),
 Row(zone_PU='Arden Heights', Average_unit_profitability_1=38.11952040113714),
 Row(zone_PU='Forest Park/Highland Park', Average_unit_profitability_1=17.84036124462736),
 Row(zone_PU=None, Average_unit_profitability_1=15.929635403944006),
 Row(zone_PU="Eltingville/Annadale/Prince's Bay", Average_unit_profitability_1=15.85021430917414),
 Row(zone_PU='Rossville/Woodrow', Average_unit_profitability_1=15.450854439425235),
 Row(zone_PU='New Dorp/Midland Beach', Average_unit_profitability_1=14.894519854525143),
 Row(zone_PU='NV', Average_unit_profitability_1=14.687994941683188),
 Row(zone_PU='Highbridge Park', Average_unit_profitability_1=13.838322471987832),
 Row(zone_PU='Queens Village', Average_unit_profitability_1=13.757510056734262)],
 [Row(zone_PU='Upper East Side South', sum_total_passengers=8265045.0),
 Row(zone_PU='Upper East Side North', sum_total_passengers=7626015.0),
 Row(zone_PU='Midtown Center', sum_total_passengers=7456960.0),
 Row(zone_PU='Penn Station/Madison Sq West', sum_total_passengers=6523834.0),
 Row(zone_PU='Midtown East', sum_total_passengers=6472875.0),
 Row(zone_PU='Times Sq/Theatre District', sum_total_passengers=6100483.0),
 Row(zone_PU='Clinton East', sum_total_passengers=5816084.0),
 Row(zone_PU='JFK Airport', sum_total_passengers=5803297.0),
 Row(zone_PU='Lincoln Square East', sum_total_passengers=5694922.0),
 Row(zone_PU='Murray Hill', sum_total_passengers=5691669.0)]]

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2,898,033 | 15,571,166 | 41,953,716 | 90,443,069 | 132,396,785 |
| execution time   (w/o 1.2)  | 12.59 seconds | 95.4 seconds | 1111.2 seconds | 1379.4 seconds | 1517.4 seconds |
| execution time              | 26.24 seconds | 237 seconds | 1981.2 seconds | 2628 seconds | 3672 seconds |
| sec / 1M records (w/o 1.2)  | 4.33  | 6.12  | 26.49 | 15.25  | 11.46  |
| sec / 1M records            | 9.05  | 15.22  | 47.22  | 29.06  | 27.73  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2,898,033 | 15,571,166 | 41,953,716 | 90,443,069 | 132,396,785 |
| execution time   (w/o 1.2)  | 11.11 seconds | 9.67 seconds | 11.68 seconds | 11.18 seconds | 29.14 seconds |
| execution time              | 14.42 seconds | 18.93 seconds | 25.11 seconds | 44.82 seconds | 78.6 seconds |
| sec / 1M records (w/o 1.2)  | 3.83  | 0.62  | 0.27 | 0.12 | 0.22  |
| sec / 1M records            | 4.97 | 1.21 | 0.59  | 0.49  | 0.59  |